In [2]:
import h5py
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras as kr
import tensorflow as tf

# Locally get vars to accommodate different environments
try:
    from env_vars import *  
except:
    print("No env_vars file found. Using default values. Make a \"env_vars.py\" file to change them.")
    intra_train_path = "your path here"
    intra_test_path  = "your path here"

In [3]:
#merge all files into one dataset
import os
import h5py
import numpy as np

def get_dataset_name(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]
    temp = filename_without_dir.split("_")[:-1]
    datasetname = "_".join(temp)
    return datasetname

def get_label(filename):
    if 'rest' in filename:
        label = 0
    elif 'math' in filename:
        label = 1
    elif 'memory' in filename:
        label = 2
    elif 'motor' in filename:
        label = 3
    return label

def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, 'r') as f:
                dataset_name = get_dataset_name(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)
              

    return dataset, labels


In [4]:

dataset = get_all_matrices(intra_train_path)[0]
# subjects x sensors x time
dataset_X = np.stack(dataset) 
labels = get_all_matrices(intra_train_path)[1]

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset_X = []

for data in dataset_X:
    scaler.fit(data)
    scaled_data = scaler.transform(data)
    scaled_dataset_X.append(scaled_data)

trainX = np.stack(scaled_dataset_X)

print(trainX)
trainY = labels


[[[0.74740712 0.74772078 0.74885491 ... 0.73938871 0.73718579 0.73627912]
  [0.40877146 0.40968423 0.4112499  ... 0.38505688 0.38551707 0.38728114]
  [0.7009247  0.70096807 0.70151863 ... 0.71032065 0.70880436 0.70774143]
  ...
  [0.774183   0.77379063 0.7753733  ... 0.77593174 0.77662985 0.77914528]
  [0.43186421 0.43222923 0.43327232 ... 0.41756787 0.41650519 0.41809432]
  [0.54323608 0.54463592 0.54744894 ... 0.53403411 0.53460681 0.53576734]]

 [[0.94303924 0.9417854  0.93858264 ... 0.93988644 0.94183427 0.94208164]
  [0.90332562 0.90380797 0.90315913 ... 0.89520783 0.89568684 0.89679132]
  [0.96081808 0.96108471 0.95981947 ... 0.96375179 0.96319005 0.96258611]
  ...
  [0.95199669 0.95089824 0.9487248  ... 0.95780987 0.95791452 0.95732887]
  [0.94888014 0.94871092 0.9471317  ... 0.94356784 0.94394967 0.94381337]
  [0.89620272 0.89621608 0.89675431 ... 0.92034025 0.92086643 0.9212247 ]]

 [[0.96028099 0.96041334 0.96033721 ... 0.96569543 0.96579696 0.96538085]
  [0.96302199 0.963033

In [5]:
dataset = get_all_matrices(intra_test_path)[0]
# subjects x sensors x time
dataset_X = np.stack(dataset) 
labels = get_all_matrices(intra_test_path)[1]

testX = dataset_X
print(testX.shape)
testY = labels


(8, 248, 35624)


In [6]:
def getdatasetname(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]
    temp = filename_without_dir.split("_")[:-1]
    datasetname = "_".join(temp)
    return datasetname

In [7]:
with h5py.File(intra_train_path, "r") as f:
    dataset_name = getdatasetname(intra_train_path)
    matrix = f.get(dataset_name)[()]
    print(type(matrix))
    print(matrix.shape)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/stijn/Desktop/Final Project data/Intra/test/rest_105923_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(matrix)
scaled_data = scaler.transform(matrix)
print(scaled_data.shape)

(248, 35624)


Downsampling


In [ ]:
print(scaled_data.shape[1] / 2034)

17.514257620452312


In [ ]:
# Calculate the factors of the sample rate 2034
import math

factors = []
for x in range(1, 2034):
    if 2034 % x == 0:
        factors.append(x)

factors = np.array(factors)

print(factors)

[   1    2    3    6    9   18  113  226  339  678 1017]


In [ ]:
# Each one of the factors can be used as a subsample size for the current sample rate of 2034.
# You can specify a freqency (n) for the data you want to keep.
# Each factor has a limit for (n), after which the subsample would be the same size as the sample

In [ ]:
min_sample_freqency = 2034 / factors
np.set_printoptions(suppress=True)


print(min_sample_freqency)

[2034. 1017.  678.  339.  226.  113.   18.    9.    6.    3.    2.]


In [ ]:
# A new sample frequency can be specified using factors of the original frequency 2034
# A frequency of 1017 would mean every second value is kept i.e., 2034/1017 = 2
# The lower the factor, the less data will be kept from the sample e.g., 2034/113 = 18 (Every 18th value will be kept)


def donwsample(dataset, frequency):
    downsampled_dataset = []

    for i in range(0, dataset.shape[2], 2034):
        second = dataset[:, i : i + 2034]
        subsample = []

        for j in range(0, 2034, int(2034 / frequency)):
            if j < second.shape[2]:
                measurement = second[:, j]
                subsample.append(measurement)

        downsampled_dataset.extend(subsample)

    return np.array(downsampled_dataset).T

In [ ]:

trainx = trainX[:, :, 0::4]
trainy = kr.utils.to_categorical(trainY)

testx = testX[:, :, 0::4]
testy = kr.utils.to_categorical(testY)

print(testx.shape)
print(testy)


(8, 248, 8906)
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [2]:
from sklearn.metrics import confusion_matrix

model = tf.keras.models.Sequential()
model.add(kr.layers.LSTM(248, input_shape=(trainx.shape[1],trainx.shape[2])))
model.add(kr.layers.Dropout(0.5))
model.add(kr.layers.Dense(248, activation='relu'))
model.add(kr.layers.Dense(trainy.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
verbose, epochs, batch_size = 0, 30, 64

predictions = model.predict(testx)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(testy, axis=1)
model.fit(trainx, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

cm = confusion_matrix(true_labels, predicted_labels)
print(cm)
_, accuracy = model.evaluate(testx, testy, batch_size=batch_size, verbose=0)
print(accuracy)

NameError: name 'tf' is not defined